In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.utils import shuffle
import time
from itertools import chain, combinations, permutations
from IPython.display import Image, display
import urllib.request

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from sklearn.pipeline import Pipeline

# Challenge: Neural network regressor to predict photometric redshifts
In observational astronomy, measuring the redshift of a galaxy tells us how far away it is, and thus provides crucial information about the structure and evolution of the universe.

Traditionally, redshifts are measured using spectroscopy, which provides highly accurate values but is expensive and time-consuming — suitable only for a limited number of objects.

To overcome this, astronomers often use photometric redshifts (photo-z): estimates of a galaxy’s redshift derived from its brightness (magnitude) in a few broad photometric bands (e.g., ugriz, JHK). These estimates are much faster to obtain, but less precise.

Goal: Build a supervised regression model that predicts the redshift of galaxies from their photometric features using a neural network. Use the spectroscopic redshift 'z' as your label for the supervision of the model.

In [3]:
url = 'https://www.researchgate.net/profile/Yuming-Fu/publication/344192975/figure/fig1/AS:993821525565440@1613956924645/Photometric-redshift-obtained-with-XGBoost-regression-model-against-spectral-redshift-of.png'

In [4]:
display(Image(url=url))
## Example figure!!! from Yuming Fu et al 2021 ApJS 254 6 (this is not what you will reproduce, but quite similar)

# A) The data

The data is based on the Sloan Digital Sky Survey (SDSS) DR16. Each row is a galaxy, and it contains 66 features + the spectroscopic redshift which is our label (supervised learning)
The features include:

- u,g,r,i,z: magnitudes of the objects in the SDSS. These represent the fluxes of light measured in logarithmic units, through various bands from the ultraviolet to the infrared
- 'ra' (right ascension) and 'dec' (declination): position of the object in the sky
- expRad: radius of the object
- expRadErr: error in the radius estimate
- 'z': spectroscopic redshift (or true redshift)

Please first download the training set from this link: https://drive.google.com/file/d/1C0cgPM2WkoB7m9eTYNftM3OGUO7Aqaee/view?usp=sharing

In [ ]:
training = pd.read_csv('DR16_trainingset.csv')
validation = pd.read_csv('DR16_validationset.csv')
testing = pd.read_csv('DR16_testingset.csv')

In [ ]:
training.keys()

#### Answer the following questions:

- The data is already split into training, test, and validation sets. What percentage of the total corresponds to training, validation, and test?

- Make a histogram of the total spectroscopic redshift 'z', as well as individual histograms of 'z' for the training, validation, and test sets. Use a log scale on the y-axis. Are these distributions similar? Do you think the splitting into training/validation/test sets was done correctly?

- Generate x_train, y_train, x_test, y_test, x_valid, y_valid. Apply a StandardScaler preprocessing step.

## B) Let's train a neural network!
The following cell provides a baseline neural network architecture. Based on this, try the following modifications:

- Add more layers and/or increase the number of neurons.

- Include additional Dropout layers between Dense layers.

- Remove all Dropout layers.

- Change the learning rate to 0.1.

Observe the effect on the training and validation loss curves for each case.

Additionally, how does the model perform at high redshift values (e.g., z > 3)?

In [ ]:
model = keras.Sequential(
    [
        keras.layers.Input(shape=(66,)),
        keras.layers.Dense(30, activation='relu'),
        keras.layers.Dropout(0.1),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='relu')
    ]
)
model.summary()
model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['mae']
    )


start = time.time()

## first define x_train, y_train, x_test, y_test, x_valid, y_valid

history = model.fit(x_train,y_train,validation_data=(x_valid, y_valid), batch_size=100, epochs=100, verbose = 1)
plt.plot(history.history['loss'],c='r',label='Training')
plt.plot(history.history['val_loss'],c='g',label='Validation')
plt.title('MAE')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend()
plt.show()

print('Testing \n')

y_pred = model.predict(x_test)
plt.scatter(y_test,y_pred,alpha=0.4,marker=r'$\clubsuit$',c='g')
plt.xlabel('$z_{spec}$')
plt.ylabel('$z_{phot}$')
plt.show()

rms = np.sqrt(np.mean((np.array(pd.DataFrame(y_pred)[0]) - y_test)** 2))
print('RMS Testing: ', rms)
print('MSE Testing: ', mean_squared_error(y_pred, y_test))
print('R2 Testing: ', r2_score(y_test, y_pred))

print('Time: ', time.time()- start)
